In [1]:
import re
import numpy as np
import pandas as pd

# visualisation
import matplotlib.pyplot as plt

# tokenizer and lemmatizer
import nltk

# sklearn
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [2]:
# If nltk does not already include those packages
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\guomu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\guomu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\guomu\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
# Change text like "Ohhhhhh my gosh!!!!!!!!" to "Ohhh my gosh!!!"
def remove_repetitions(text, nb_repetition = 3):
    out = ""
    last = ''
    count = 0
    for c in text:
        if c == last:
            count += 1
        else:
            count = min(count, nb_repetition)
            for i in range(min(count, nb_repetition)):
                out += last
            last = c
            count = 1
    for i in range(min(count, nb_repetition)):
                out += last
    return out

In [4]:
# Preprocessing of text
def clean_data(data):
    for i in range(len(data)):
        # Remove capital letters 
        data[i] = data[i].lower()
        
        # Remove links
        data[i] = re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+','',data[i])
        
        # Replace @some_user to simply @ (ignore name of user but keep fact that used an @)
        data[i] = re.sub('@(\S)*', '@', data[i])
        
        #Remove repetition of characters
        data[i] = remove_repetitions(data[i])
        
        #Consider "..." as useful word
        data[i] = re.sub('\.{3}', ' threedots ', data[i])
        
        #Remove apostrophe 
        data[i] = re.sub('\'', '', data[i])
                
        #Remove sequences like "&quot;" and characters aside from letters (without accents) aside from "@"
        data[i] = re.sub('&.*;|[^a-zA-z@\s]', '', data[i])
        
    return data
        

In [5]:
# Change textual label to int
# 0: negative ; 1: neutral ; 2: positive
def label_to_int(labels):
    labels_int = np.zeros(labels.shape,dtype='int')
    for i in range(len(labels)):
        if labels[i] == 'neutral':
            labels_int[i] = 1
        elif labels[i] == 'positive':
            labels_int[i] = 2
    return labels_int
    

In [14]:
# Calls nltk's tokenizer on array of entries
def tokenize(data):
    for i in range(len(data)):
        data[i] = np.array(nltk.tokenize.word_tokenize(data[i]))
    return data

In [15]:
# Calls nltk's lemmatizer on array of entries already lemmatized
def lemmatize(data):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    for i in range(len(data)):
        for j in range(len(data[i])):
            data[i][j] = lemmatizer.lemmatize(data[i][j])
    return data

In [159]:
# Inspired by https://www.jmlr.org/papers/volume2/lodhi02a/lodhi02a.pdf
# Decay is a hyperparameter between 0 and 1 controlling decay when sequences are far in the text
# When decay = 1, "cat" and "cut" would have the same score as "cat" and "ct"
# "n" specifies the size of subsequences to consider (not the max size, but the exact size)
def string_subsequence_kernel(sequence1, sequence2, decay=0.03, n=5):
    
    # Define s and t, the strings to compare
    s = sequence1
    t = sequence2
    
    # dynamic programming matrix for K'' (see article for details)
    # used to reduce computation cost of O(n|s||t|^2) to O(n|s||t|) (do not iterate again on |t|)
    m2 = np.empty((len(t)+1, n+1, len(s)+1))            
    
    # dynamic programming matrix with m[i][j][k] containing K'_j(s[0:k+1], t[0:i+1])
    # aside for when j = n, then it contains K_n instead of K'_n
    # cell m[len(t)][n][len(s)] contain the desired value
    # (see article for details)
    m = np.empty((len(t)+1, n+1,len(s)+1))
    
    # Iterate over length of t to consider
    for i in range(len(t)+1):
        # Iterate over n (lengh of sequence)
        for j in range(n+1):
            # Iterate over length of s to consider
            for k in range(len(s)+1):
                
                # n = 0, give score of 1
                if j == 0:
                    m[i][j][k] = 1
                    m2[i][j][k] = 1
                
                # one string is too short compared to n, score of 0
                elif k < j or i < j:
                    m[i][j][k] = 0
                    m2[i][j][k] = 0
                    
                # last row, compute K_n
                elif j == n:
                    # Add score for s[:-1] for current s length
                    m[i][j][k] = m[i][j][k-1]
                    
                    # Find subsequences in t that match character s[-1] for current s length
                    char = s[k-1]
                    
                    # Iterate over chars of t 
                    for l in range(i):
                        if t[l] == char:
                            m[i][j][k] += m[l][j-1][k-1] * decay**2
                            
                # compute K'_n
                else :
                    # Add score for s[:-1] for current s length
                    m[i][j][k] = decay * m[i][j][k-1]
                    
                    # Find subsequences in t that match character s[-1] for current s length
                    char = s[k-1]
                    
                    # Use optimization
                    if char == t[i-1]:
                        m2[i][j][k] = decay*(m2[i-1][j][k] + decay * m[i-1][j-1][k-1])                                                
                        
                    else :
                        m2[i][j][k] = decay*m2[i-1][j][k]
                    
                    m[i][j][k] += m2[i][j][k]
                    
                    # Without optimization:
                    
                    # Iterate over chars of t 
                    #for l in range(i):
                    #    if t[l] == char:
                    #        m[i][j][k] += m[l][j-1][k-1] * decay**(i - l + 1) # note that the paper uses indexes from 0 
                                
                #print(i,"   ",j, "   ",k, "   ", m[i][j][k])
                                
    return m[len(t)][n][len(s)]            

In [112]:
# Normalizes the value from string_subsequence_kernel
# If the kernel is precomputed, this function is not used to avoid recomputing the denominator for each value 
# Useful if we want to pass a callable to the SVM
def kernel_matrix_value(sequence1, sequence2, decay=0.03, n=5):
    return string_subsequence_kernel(sequence1, sequence2, decay, n)/np.sqrt(string_subsequence_kernel(sequence1, sequence1, decay, n)*string_subsequence_kernel(sequence2, sequence2, decay, n))

In [81]:
# Useful when too many data for kernel
# We could make batches to construct multiple smaller classifiers and have them vote
def divide_data(data, nb_batches):
    nb_entries = len(data)
    batch_size = nb_entries/nb_batches
    print("Received ", nb_entries, " entries, dividing into batches of around ", batch_size, " entries")
    
    batches = np.empty(int(nb_batches),dtype=object)
    
    current_pointer = 0
    for i in range(nb_batches):
        if i == nb_batches-1:
            batches[i] = data[current_pointer:]
        else:
            batch_size = int(np.round(float(nb_entries)/(nb_batches-i)))
            batches[i] = data[current_pointer:(current_pointer + batch_size)]
            nb_entries -= batch_size
            current_pointer += batch_size
    return batches

In [32]:
# Join array of text to single string
# Used after tokenization and lemmatization since string_subsequence_kernel works on strings and not array of words
def joint_data(data):
    for i in range(len(data)):
        data[i] = ' '.join(data[i])
    return data

In [ ]:
#####################################################################################################
#                                                                                                   #
#    Run this section if you DON'T already have train_data_cleaned.csv and test_data_cleaned.csv    #
#                                                                                                   #
#####################################################################################################

In [29]:
# Load data
train_data = pd.read_csv('train_data.csv')
train_label = pd.read_csv('train_results.csv')
test_data = pd.read_csv('test_data.csv')

# Preprocessing
# Drop index
train_data = train_data.values[:, -1]
test_data = test_data.values[:, -1]
train_label = train_label.values[: ,-1]

# Labels as int (0: negative; 1: neutral; 2: positive)
train_label = label_to_int(train_label)

In [30]:
# Clean data
train_data = clean_data(train_data)
test_data = clean_data(test_data)

In [31]:
# Tokenize
train_data = tokenize(train_data)
test_data = tokenize(test_data)

# Lemmatize
train_data = lemmatize(train_data)
test_data = lemmatize(test_data)


In [33]:
train_data = joint_data(train_data)
test_data = joint_data(test_data)

In [34]:
# Run only once to produce train_data_cleaned.csv
# Next time, simply load those files
df = pd.DataFrame({'id': np.arange(len(train_data)),
                      'text': train_data})
df.to_csv('train_data_cleaned.csv', index=False)

In [35]:
# Run only once to produce test_data_cleaned.csv
# Next time, simply load those files
df = pd.DataFrame({'id': np.arange(len(test_data)),
                      'text': test_data})
df.to_csv('test_data_cleaned.csv', index=False)

In [ ]:
###########################################################################################
#                                                                                         #
#    Run this section if ALREADY have train_data_cleaned.csv and test_data_cleaned.csv    #
#                                                                                         #
###########################################################################################

In [ ]:
# Load data
train_data = pd.read_csv('train_data_cleaned.csv')
train_label = pd.read_csv('train_results.csv')
test_data = pd.read_csv('test_data_cleaned.csv')

# Preprocessing
# Drop index
train_data = train_data.values[:, -1]
test_data = test_data.values[:, -1]
train_label = train_label.values[: ,-1]

# Labels as int (0: negative; 1: neutral; 2: positive)
train_label = label_to_int(train_label)

In [ ]:
############################################################################
#                                                                          #
#    Run following sections after you ran one of the precedent sections    # 
#                                                                          #
############################################################################

In [36]:
# Split train and validation
x_train, x_validation, y_train, y_validation = train_test_split(train_data, train_label, test_size=0.1)

In [120]:
# Function to use when computing the kernel for one dataset, during training
# More efficient than the computation of the kernel for test
def compute_kernel(data, decay = 0.03, n = 5):
    kernel = np.empty((len(data), len(data)))
    print("Computing kernel of ", len(data)**2, "entries")
    
    # Reusable values used in the denominator
    ssk = np.empty(len(data))
    for i in range(len(data)):
        ssk[i] = string_subsequence_kernel(data[i], data[i], decay, n)
    
    count = 0
    for i in range(len(data)):
        for j in range(len(data)):
            if i == j:
                kernel[i][j] = 1
            elif i > j:
                kernel[i][j] = kernel[j][i]
            else:
                kernel[i][j] = string_subsequence_kernel(data[i], data[j], decay, n)/np.sqrt(ssk[i]*ssk[j])
            count += 1
            if count % 1000 == 0:
                print("Computed ", count, "entries")
    return kernel

In [114]:
# Function to use when computing the kernel between two datasets, during testing
def compute_kernel_test(train, test, decay = 0.03, n = 5):
    kernel = np.empty((len(train), len(test)))
    print("Computing kernel of ", len(train)*len(test), "entries")
    
    ssk_train = np.empty(len(train))
    for i in range(len(train)):
        ssk_train[i] = string_subsequence_kernel(train[i], train[i], decay, n)
    ssk_test = np.empty(len(test))
    for i in range(len(test)):
        ssk_test[i] = string_subsequence_kernel(test[i], test[i], decay, n)
    
    count = 0
    for i in range(len(train)):
        for j in range(len(test)):            
            kernel[i][j] = string_subsequence_kernel(train[i], test[j], decay, n)/np.sqrt(ssk_train[i]*ssk_test[j])
            count += 1
            if count % 1000 == 0:
                print("Computed ", count, "entries")
    return kernel

In [116]:
# Used to test implementation, not called
def compute_self_ssk(data, decay=0.03, n=5):
    ssk = np.empty(len(data))
    for i in range(len(data)):
        ssk[i] = string_subsequence_kernel(data[i], data[i], decay, n)
    return ssk

In [44]:
# Hyperparameters :
kernel_decay = 0.03
kernel_n = 5

In [199]:
train_batches = divide_data(train_data,3000) 
test_batches = divide_data(test_data,1000)

Received  1040323  entries, dividing into batches of around  346.77433333333335  enties
Received  560175  entries, dividing into batches of around  560.175  enties


In [201]:
# Computed only for the first batch of 347 entries, due to lack of time
kernel = compute_kernel(train_batches[0])

Computing kernel of  120409 entries
Computed  1000 entries
Computed  2000 entries
Computed  3000 entries
Computed  4000 entries
Computed  5000 entries
Computed  6000 entries
Computed  7000 entries
Computed  8000 entries
Computed  9000 entries
Computed  10000 entries
Computed  11000 entries
Computed  12000 entries
Computed  13000 entries
Computed  14000 entries
Computed  15000 entries
Computed  16000 entries
Computed  17000 entries
Computed  18000 entries
Computed  19000 entries
Computed  20000 entries
Computed  21000 entries
Computed  22000 entries
Computed  23000 entries
Computed  24000 entries
Computed  25000 entries
Computed  26000 entries
Computed  27000 entries
Computed  28000 entries
Computed  29000 entries
Computed  30000 entries
Computed  31000 entries
Computed  32000 entries
Computed  33000 entries
Computed  34000 entries
Computed  35000 entries
Computed  36000 entries
Computed  37000 entries
Computed  38000 entries
Computed  39000 entries
Computed  40000 entries
Computed  410

In [202]:
# Run only once to save computation of the first kernel
np.savetxt("kernel1.csv",kernel)

In [203]:
# Test with some values 
predictions = np.empty(0)

svc = SVC(kernel='precomputed')

kernel_train = kernel

svc.fit(kernel_train, y_train[0:len(train_batches[0])])

# Would be useful if we had trained more kernels
votes = np.zeros((99,3))

# 99 test samples, the 99 last entries of train_data
kernel_test = compute_kernel_test(train_data[-100:-1], train_batches[0])

y_pred = svc.predict(kernel_test)

# Would be useful if we had trained more kernels
for k in range(len(y_pred)):
    votes[k][int(y_pred[k])] += 1
    
for k in range(len(votes)):
    predictions = np.append(predictions, np.argmax(votes[k]))
        

Computing kernel of  34353 entries
Computed  1000 entries
Computed  2000 entries
Computed  3000 entries
Computed  4000 entries
Computed  5000 entries
Computed  6000 entries
Computed  7000 entries
Computed  8000 entries
Computed  9000 entries
Computed  10000 entries
Computed  11000 entries
Computed  12000 entries
Computed  13000 entries
Computed  14000 entries
Computed  15000 entries
Computed  16000 entries
Computed  17000 entries
Computed  18000 entries
Computed  19000 entries
Computed  20000 entries
Computed  21000 entries
Computed  22000 entries
Computed  23000 entries
Computed  24000 entries
Computed  25000 entries
Computed  26000 entries
Computed  27000 entries
Computed  28000 entries
Computed  29000 entries
Computed  30000 entries
Computed  31000 entries
Computed  32000 entries
Computed  33000 entries
Computed  34000 entries


In [213]:
# Just to verify that it "seems" to work
predictions[0:99]

array([0., 0., 0., 2., 0., 2., 0., 0., 2., 0., 2., 0., 0., 2., 2., 0., 0.,
       0., 0., 0., 0., 2., 2., 0., 0., 2., 0., 0., 2., 0., 0., 0., 0., 0.,
       0., 2., 0., 2., 2., 0., 0., 0., 0., 0., 2., 0., 0., 2., 0., 2., 0.,
       0., 0., 2., 0., 2., 0., 0., 0., 2., 0., 0., 0., 2., 2., 0., 2., 0.,
       2., 2., 0., 0., 2., 2., 0., 0., 0., 0., 0., 0., 2., 0., 0., 2., 2.,
       0., 0., 0., 0., 0., 0., 0., 0., 2., 2., 0., 0., 0., 2.])

In [215]:
saveResults(predictions[0:99])

In [214]:
# Compare with real entries
y_train[-100:-1]

array([0, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 2,
       0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2,
       2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 2, 2,
       0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0,
       2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 0])

In [207]:
# Compte error rate
def test(predictions, labels):
        nb_errors = 0.0
        for i in range(len(labels)):            
            if predictions[i] != labels[i]:
                nb_errors += 1.0
                
        return nb_errors/len(labels)

In [212]:
# Evaluate
test(predictions[0:99], y_train[-100:-1])


0.5252525252525253

In [ ]:
###############################################################
#                                                             #
#   Code below this point were not used but could be useful   #
#                                                             #
###############################################################

In [106]:
#just 1 kernel
predictions = np.empty(0)

svc = SVC(kernel='precomputed')

kernel_train = kernel

svc.fit(kernel_train, y_train[0:len(train_batches[0])])

# Calculate for some batches of test
for i in range(10):
    print("Batch", i)
    votes = np.zeros((len(test_batches[i]),3))
    kernel_test = compute_kernel_test(test_batches[i], train_batches[0])
        
    y_pred = svc.predict(kernel_test)
    for k in range(len(y_pred)):
        votes[k][int(y_pred[k])] += 1
    
for k in range(len(votes)):
    predictions = np.append(predictions, np.argmax(votes[k]))
        

Batch 0
Computing kernel of  5824 entries


C:\Users\guomu\AppData\Local\Temp/ipykernel_87760/2261394038.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  kernel[i][j] = string_subsequence_kernel(train[i], test[j], decay, n)/np.minimum(np.sqrt(ssk_train[i]*ssk_test[j]),1e-4)
C:\Users\guomu\AppData\Local\Temp/ipykernel_87760/2261394038.py:15: RuntimeWarning: invalid value encountered in double_scalars
  kernel[i][j] = string_subsequence_kernel(train[i], test[j], decay, n)/np.minimum(np.sqrt(ssk_train[i]*ssk_test[j]),1e-4)


Computed  1000 entries
Computed  2000 entries
Computed  3000 entries
Computed  4000 entries
Computed  5000 entries


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [77]:
# For all
# Would take to long to run unfortunately
counter_train = 0
predictions = np.empty(0)
for i in range(len(test_batches)):
    votes = np.zeros((len(test_batches[i]),3))
    kernel_test = compute_kernel(test_batches[i])
    for j in range(len(train_batches)):
        print("Calculations for batch train ", j, ", and test ", i)
        kernel_train = compute_kernel(train_batches[j])
        svc = SVC(kernel='precomputed')
        
        svc.fit(kernel_train, y_train[counter_train:(counter_train + len(train_batches[j]))])
        counter_train += len(train_batches[j])
        y_pred = svc.predict(kernel_test)
        for k in range(len(y_pred)):
            votes[k][int(y_pred[k])] += 1
    
for k in range(len(votes)):
    predictions = np.append(predictions, np.argmax(votes[k]))       


Computing kernel of  2179115761 entries


C:\Users\guomu\AppData\Local\Temp/ipykernel_87760/215594344.py:3: RuntimeWarning: invalid value encountered in double_scalars
  return string_subsequence_kernel(sequence1, sequence2, decay, n)/np.sqrt(string_subsequence_kernel(sequence1, sequence1, decay, n)*string_subsequence_kernel(sequence2, sequence2, decay, n))


KeyboardInterrupt: 

In [37]:
# Function to save the results as predictions.csv
def saveResults(predictions):
    df = pd.DataFrame({'id': np.arange(len(predictions)),
                      'target': predictions})
    df.astype(int).to_csv('predictions.csv', index=False)

In [ ]:
###################################################################################################
#                                                                                                 #
#    Below are some old code that were used to find best hyperparameter for logistic regression   #
#    Remove for final submission                                                                  #
#                                                                                                 #
###################################################################################################

In [ ]:
# Train for each hyperparameter
#errors = np.empty(len(FIRST_HYPERPARAMETER))
#print("Training for SVM")
#for i in range(len()):
#    val = regularization_terms[i]
#    print("Current hyperparameter value :", str(val))
#    logistic_regression = Logistic_Regression(np.zeros((nb_features + 1, NB_CLASSES)), reg, np.arange(NB_CLASSES))
#    logistic_regression.train(x_train, y_train, initial_step=initial_step, max_steps=step_number, plt=False)
#    errors[i] = logistic_regression.test(x_validation, y_validation)

# Choose best hyperparameter
#index_min = np.argmin(errors)
#best_hyperparam = ARRAY_OF_HYPERPARAM[index_min]
#print("Best value for hyperparameter : ", str(best_hyperparam))
#print("Error on validation for best hyperparameter", str(errors[index_min]))

In [ ]:
# Results for all hyperparameters
#print("Values tested :", regularization_terms)
#print("Error rates :", errors)

In [ ]:
# Train and obtain predictions for best hyperparameter
#logistic_regression = Logistic_Regression(np.zeros((nb_features + 1, NB_CLASSES)), best_lambda, np.arange(NB_CLASSES))
#l,e = logistic_regression.train(np.concatenate((x_train, x_validation)), np.concatenate((y_train, y_validation)), initial_step=initial_step, max_steps=step_number, plt=False)
#pred = logistic_regression.predict(test_data)

In [ ]:
# Save results of predictions on test data
#saveResults(pred)